In [0]:
%sql
WITH base_cnos AS (
  -- Query 1 base: use C-NOS
  SELECT
    category,
    quarter,
    month,
    CAST(SUBSTRING(month, 6, 2) AS INT) AS month_num,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) AS cnos,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(target           AS DOUBLE) END) AS cnos_target,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(ya_actual        AS DOUBLE) END) AS cnos_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month >= '2025M07' AND month <= '2025M12'
    AND quarter IN ('JAS','OND')
  GROUP BY category, quarter, month
),

-- ---------- Per-category rollups (C-NOS) ----------
quarter_rollup AS (
  SELECT
    category,
    quarter,
    SUM(cnos)        AS q_cnos,
    SUM(cnos_target) AS q_target,
    SUM(cnos_ya)     AS q_ya
  FROM base_cnos
  GROUP BY category, quarter
),

half_year AS (
  SELECT
    category,
    SUM(cnos)        AS hy_cnos,
    SUM(cnos_target) AS hy_target,
    SUM(cnos_ya)     AS hy_ya
  FROM base_cnos
  GROUP BY category
),

month_wide AS (
  SELECT
    category,
    MAX(CASE WHEN month = '2025M07' THEN cnos END)/1000/1000/150.1 AS JULY_MM,
    MAX(CASE WHEN month = '2025M07' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS JULY_IYA,
    MAX(CASE WHEN month = '2025M08' THEN cnos END)/1000/1000/150.1 AS AUG_MM,
    MAX(CASE WHEN month = '2025M08' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS AUG_IYA,
    MAX(CASE WHEN month = '2025M09' THEN cnos END)/1000/1000/150.1 AS SEP_MM,
    MAX(CASE WHEN month = '2025M09' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS SEP_IYA,
    MAX(CASE WHEN month = '2025M10' THEN cnos END)/1000/1000/150.1 AS OCT_MM,
    MAX(CASE WHEN month = '2025M10' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS OCT_IYA,
    MAX(CASE WHEN month = '2025M11' THEN cnos END)/1000/1000/150.1 AS NOV_MM,
    MAX(CASE WHEN month = '2025M11' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS NOV_IYA,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/1000/1000/150.1 AS 1H_to_date,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos_ya END),0)*100 AS 1H_to_date_IYA
  FROM base_cnos
  GROUP BY category
),

category_metrics AS (
  SELECT
    m.category,
    -- Quarterly KPIs (C-NOS)
    MAX(CASE WHEN q.quarter = 'JAS' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS JAS_IYA,
    MAX(CASE WHEN q.quarter = 'OND' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS OND_IYA,
    (h.hy_target)/1000/1000/150.1                                             AS 1H_MM,
    (h.hy_target/NULLIF(h.hy_ya,0))*100                                       AS 1H_IYA,
    -- Monthly + To-Date (C-NOS)
    m.JULY_MM         AS JUL_MM,
    m.JULY_IYA        AS JUL_IYA,
    m.AUG_MM          AS AUG_MM,  
    m.AUG_IYA         AS AUG_IYA,
    m.SEP_MM          AS SEP_MM,  
    m.SEP_IYA         AS SEP_IYA,
    m.OCT_MM          AS OCT_MM,  
    m.OCT_IYA         AS OCT_IYA,
    m.NOV_MM          AS NOV_MM,  
    m.NOV_IYA         AS NOV_IYA,
    m.1H_to_date      AS 1H_To_Date_MM,
    m.1H_to_date_IYA  AS 1H_To_Date_IYA
  FROM month_wide m
  LEFT JOIN quarter_rollup q ON q.category = m.category
  LEFT JOIN half_year      h ON h.category = m.category
  GROUP BY m.category, h.hy_target, h.hy_cnos, h.hy_ya,
           m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
           m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA,m.NOV_MM,m.NOV_IYA,
           m.1H_to_date, m.1H_to_date_IYA
),

-- ---------- TOTAL rollup (C-NOS) ----------
t_quarter AS (
  SELECT
    'TOTAL' AS category,
    quarter,
    SUM(cnos)        AS q_cnos,
    SUM(cnos_target) AS q_target,
    SUM(cnos_ya)     AS q_ya
  FROM base_cnos
  GROUP BY quarter
),

t_half_year AS (
  SELECT
    'TOTAL' AS category,
    SUM(cnos)        AS hy_cnos,
    SUM(cnos_target) AS hy_target,
    SUM(cnos_ya)     AS hy_ya
  FROM base_cnos
),

t_month_wide AS (
  SELECT
    'TOTAL' AS category,
    SUM(CASE WHEN month = '2025M07' THEN cnos END)/1000/1000/150.1 AS JULY_MM,
    SUM(CASE WHEN month = '2025M07' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M07' THEN cnos_ya END),0) * 100 AS JULY_IYA,
    SUM(CASE WHEN month = '2025M08' THEN cnos END)/1000/1000/150.1 AS AUG_MM,
    SUM(CASE WHEN month = '2025M08' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M08' THEN cnos_ya END),0) * 100 AS AUG_IYA,
    SUM(CASE WHEN month = '2025M09' THEN cnos END)/1000/1000/150.1 AS SEP_MM,
    SUM(CASE WHEN month = '2025M09' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M09' THEN cnos_ya END),0) * 100 AS SEP_IYA,
    SUM(CASE WHEN month = '2025M10' THEN cnos END)/1000/1000/150.1 AS OCT_MM,
    SUM(CASE WHEN month = '2025M10' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M10' THEN cnos_ya END),0) * 100 AS OCT_IYA,
    SUM(CASE WHEN month = '2025M11' THEN cnos END)/1000/1000/150.1 AS NOV_MM,
    SUM(CASE WHEN month = '2025M11' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M11' THEN cnos_ya END),0) * 100 AS NOV_IYA,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/1000/1000/150.1 AS 1H_to_date,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos_ya END),0)*100 AS 1H_to_date_IYA
  FROM base_cnos
),

total_metrics AS (
  SELECT
    m.category,
    MAX(CASE WHEN q.quarter = 'JAS' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS JAS_IYA,
    MAX(CASE WHEN q.quarter = 'OND' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS OND_IYA,
    (h.hy_target)/1000/1000/150.1                                             AS 1H_MM,
    (h.hy_target/NULLIF(h.hy_ya,0))*100                                       AS 1H_IYA,
    m.JULY_MM         AS JUL_MM,
    m.JULY_IYA        AS JUL_IYA,
    m.AUG_MM          AS AUG_MM,  
    m.AUG_IYA         AS AUG_IYA,
    m.SEP_MM          AS SEP_MM,  
    m.SEP_IYA         AS SEP_IYA,
    m.OCT_MM          AS OCT_MM,  
    m.OCT_IYA         AS OCT_IYA,
    m.NOV_MM          AS NOV_MM,  
    m.NOV_IYA         AS NOV_IYA,
    m.1H_to_date      AS 1H_To_Date_MM,
    m.1H_to_date_IYA  AS 1H_To_Date_IYA
  FROM t_month_wide m
  LEFT JOIN t_quarter q ON q.category = m.category
  LEFT JOIN t_half_year h ON h.category = m.category
  GROUP BY m.category, h.hy_target, h.hy_cnos, h.hy_ya,
           m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
           m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA,m.NOV_MM,m.NOV_IYA,
           m.1H_to_date, m.1H_to_date_IYA
),

-- ---------- logic for GIV ----------
giv_category AS (
  SELECT
    category,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv_ya END),0) * 100
      AS GIV_1H_to_date_IYA
  FROM (
    SELECT
      category,
      month,
      SUM(CASE WHEN account = 'GIV' THEN CAST(forecast_act_est AS DOUBLE) END) AS giv,
      SUM(CASE WHEN account = 'GIV' THEN CAST(ya_actual        AS DOUBLE) END) AS giv_ya
    FROM hive_metastore.japan_gold.cnos_vw
    WHERE month >= '2025M07' AND month <= '2025M12'
    GROUP BY category, month
  ) s
  GROUP BY category
),

giv_total AS (
  SELECT
    'TOTAL' AS category,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv_ya END),0) * 100
      AS GIV_1H_to_date_IYA
  FROM (
    SELECT
      month,
      SUM(CASE WHEN account = 'GIV' THEN CAST(forecast_act_est AS DOUBLE) END) AS giv,
      SUM(CASE WHEN account = 'GIV' THEN CAST(ya_actual        AS DOUBLE) END) AS giv_ya
    FROM hive_metastore.japan_gold.cnos_vw
    WHERE month >= '2025M07' AND month <= '2025M12'
    GROUP BY month
  ) s
),

giv_union AS (
  SELECT * FROM giv_category
  UNION ALL
  SELECT * FROM giv_total
),

final_union AS (
  SELECT
    f.*,
    g.GIV_1H_to_date_IYA
  FROM (
    SELECT * FROM category_metrics
    UNION ALL
    SELECT * FROM total_metrics
  ) f
  LEFT JOIN giv_union g
    ON g.category = f.category
)

-- ---------- Final output ----------
SELECT
  category,
    ROUND(JAS_IYA, 0) AS JAS_IYA,
    ROUND(OND_IYA, 0) AS OND_IYA,
    ROUND(`1H_MM`, 0) AS `1H_MM`,
    ROUND(`1H_IYA`, 0) AS `1H_IYA`,
    ROUND(JUL_MM, 0) AS JUL_MM,
    ROUND(JUL_IYA, 0) AS JUL_IYA,
    ROUND(AUG_MM, 0) AS AUG_MM,
    ROUND(AUG_IYA, 0) AS AUG_IYA,
    ROUND(SEP_MM, 0) AS SEP_MM,
    ROUND(SEP_IYA, 0) AS SEP_IYA,
    ROUND(OCT_MM, 0) AS OCT_MM,
    ROUND(OCT_IYA, 0) AS OCT_IYA,
    ROUND(NOV_MM, 0) AS NOV_MM,
    ROUND(NOV_IYA, 0) AS NOV_IYA,
    ROUND(1H_To_Date_MM, 0) AS 1H_To_Date_MM,
    ROUND(1H_To_Date_IYA, 0) AS 1H_To_Date_IYA,
  ROUND(`1H_To_Date_IYA`-`1H_IYA` , 0) AS `vs_Target`,      

  ROUND(GIV_1H_to_date_IYA, 0) AS `GIV_1H_to_date_IYA`,            

  ROUND(`1H_To_Date_IYA` - GIV_1H_to_date_IYA, 0) AS `Gap_VS._C-NOS`  
   

FROM final_union
ORDER BY
  CASE category
    WHEN 'TOTAL'           THEN 0
    WHEN 'Detergent'       THEN 1
    WHEN 'Fabric Enhancer' THEN 2
    WHEN 'Dish'            THEN 3
    WHEN 'Air'             THEN 4 
    WHEN 'Baby'            THEN 5 
    WHEN 'Fem'             THEN 6
    WHEN 'Hair'            THEN 7
    WHEN 'Shave'           THEN 8
    WHEN 'Oral'            THEN 9
    WHEN 'Braun'           THEN 10
    WHEN 'POC'             THEN 11
    ELSE 99
  END ASC;